In [74]:
from Crypto.Util.number import long_to_bytes

In [75]:
def short_pad_attack(c1, c2, e, n, a):
    PRxy.<x,y> = PolynomialRing(Zmod(n))
    PRx.<xn> = PolynomialRing(Zmod(n))
    PRZZ.<xz,yz> = PolynomialRing(Zmod(n))
    g1 = x ^ e - c1
    g2 = (a * x + y) ^ e - c2
    q1 = g1.change_ring(PRZZ)
    q2 = g2.change_ring(PRZZ)
    h = q2.resultant(q1)
    h = h.univariate_polynomial()
    h = h.change_ring(PRx).subs(y=xn)
    h = h.monic()
    kbits = n.nbits()//(2 * e * e)
    diff = h.small_roots(X=2^kbits, beta=0.4)[0]  # find root < 2^kbits with factor >= n^0.4
    return diff

In [76]:
def franklin_reiter_attack(c1, c2, e, n, a, b):
    PRx.<x> = PolynomialRing(Zmod(n))
    g1 = x^e - c1
    g2 = (a * x + b)^e - c2
    def gcd(g1, g2):
        while g2:
            g1, g2 = g2, g1 % g2
        return g1.monic()
    return -gcd(g1, g2)[0]

In [77]:
n = 20361372240024088786698455948788052559208001789410016096382703853157107986024860262721685000417719260611935731634077852127432140361792767202581631816544546972750034494061276779878409544779707914261679633764772575040304712361634318086289783951555842021028438799649252652041211341825451500751760872572402250747982495384263677669526575825183733353800694161425360299521143726681387485097281832219009682768523304737252763907939642212542959846630464628135025203489075698699980715986689341069964387779523254203021424865355054215122316160201073604105317768112281914334065349420946717116563634883368316247495042216330408372176714499012778410160478384503335610321108263706243329745785632599707740534386988945259578897614317582546751658480917188464178997026284336861027299289073045677754342746386408505695243800685323283852020325044649604548575089927541935884800327121875191739922436199496098842684301207745090701158839031935190703347091
e = 3
c1 = 3639847731266473012111996909765465259684540134584180368372338570948892196816095838781423020996407457408188225238520927483809091079993151555076781372882518810174687150067903870448436299501557380508793238254471833275507634732947964907461619182112787911133054275872120243558556697900528427679352181961312958660881800731678134481664074711076672290178389996403357076809805422591851145306425951725627843352207233693810474618882394140691334742086008967260117740486955640068190440609984095657695423536016475468229419187489359563800737261212975921663803729112420222039005478830477455592167092520074509241894829304209406713781082959299623674294927249556083486223036858674077173104518013601628447504500606447821540687465361616447631579976579754996021653630804073535352129315413118764836270751250405649683786487251823247828947202336680538849571498780353357272103697510910576879383751704763858882439578045020243015928994208017750848637513
c2 = 7030777127779173206633582847346001157991477456002191926122836599155148909465054067800807615361108442560942058865403188672629297039703065927801771646334817871335134889139894648729527452541098449842202838983982508551750669662540615534327150829869964429006130891731472099912937717406120443380283548571270317421722042835639732966975812764084015221255115940508456442279902250677665136380988902682370875602145833135937210740790528756301051981994351553247852018355526641012434670664732924491790949235519600899289515495046353559475806935200029321563549553167235419039924276406059858659476329718809657072997385947262654743181242885709558209249589482036673428723035300722280229192727192487772217518673838209646300548275957450994828221329299666216457961746189885356929698674294944243729739850927111231235060005119781652245234537583181232715964191675241206562888107252569566488402724441835466680342239244581162530424964324562530832713397
a = 5

# 注意
如果未提供b则可尝试通过short pad attack获取b的值

In [78]:
b = short_pad_attack(c1, c2, e, n, a)
print(b)

714


In [79]:
m1 = franklin_reiter_attack(c1, c2, e, n, a, b)
m2 = a * m1 + b
print(m1)
print(m2)

153825288616407542734919511160359014401888182850314472769290928486517268729935022247316737053250308073746472804093669693357223174200482626309533529675828482194838951112584376667097776211803735802734163923995423953464212795474099182334274189325134694595476369758794997842329418616111329225615979898403815097817
769126443082037713674597555801795072009440914251572363846454642432586343649675111236583685266251540368732364020468348466786115871002413131547667648379142410974194755562921883335488881059018679013670819619977119767321063977370495911671370946625673472977381848793974989211647093080556646128079899492019075489799


In [83]:
import gmpy2

c = 121836624300974075697021410307617877799398704636412997043885070081959280989429720121505939271618801519845564677294487289085261071864489530938936756975266796724602572135614554790383740417604947122325421381322155502222532570899845171858215244411945889235509975121332503672838693190271397334662495169940649349725607212867270114445618201171582223868214171942753939282404133460110489725340075179818856587044172460703519751189284498768640898837525773823127259807337383870535232880471869465188882667401540052151795173003568424369575866780354852158304748299284900468768898966143729562589110027789165774068500360970335261801131264801996703446527156709491597639262305131309592217711956181866054589085773085822482247966030763162382493197473555330201343835684065991963179440335668817727280429581864224497755004825170263803174390985868997862117983334405815543271969716910040927833496696049703621334172902517666284662473059140662717708823
e = 65537
n = 20361372240024088786698455948788052559208001789410016096382703853157107986024860262721685000417719260611935731634077852127432140361792767202581631816544546972750034494061276779878409544779707914261679633764772575040304712361634318086289783951555842021028438799649252652041211341825451500751760872572402250747982495384263677669526575825183733353800694161425360299521143726681387485097281832219009682768523304737252763907939642212542959846630464628135025203489075698699980715986689341069964387779523254203021424865355054215122316160201073604105317768112281914334065349420946717116563634883368316247495042216330408372176714499012778410160478384503335610321108263706243329745785632599707740534386988945259578897614317582546751658480917188464178997026284336861027299289073045677754342746386408505695243800685323283852020325044649604548575089927541935884800327121875191739922436199496098842684301207745090701158839031935190703347091
p = int(m1)
q = int(m2)

In [85]:
r = n // (p * q)
phi = (p - 1) * (q - 1) * (r - 1)
d = gmpy2.invert(gmpy2.mpz(e), gmpy2.mpz(phi))
m = pow(c, d, n)
print(long_to_bytes(m))

b'flag{d3cada43775097d86a7daec938af3c73}'
